<a href="https://colab.research.google.com/github/FrancescoZanella/Neural-networks---Plant-species-classification/blob/main/CFM_def.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/My Drive/CHALLENGE

In [ ]:
!pip install keras_tuner -q

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import tensorflow_datasets as tfds
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import keras_tuner
hp = keras_tuner.HyperParameters()
hp1 = keras_tuner

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
!unzip training_dataset_homework1.zip

# Data Loading and Preprocessing

In [ ]:
BATCH_SIZE = 256
IMG_SIZE = 96

In [ ]:
class_names = [
    "Species1",
    "Species2",
    "Species3",
    "Species4",
    "Species5",
    "Species6",
    "Species7",
    "Species8"
]
# using
train_ds = tf.keras.utils.image_dataset_from_directory(
    "training_data_final",
    labels = "inferred", # means that labels are retrieved automaticallly with files name
    label_mode = "categorical",
    color_mode = "rgb",
    batch_size = BATCH_SIZE,
    image_size = (96,96),
    seed = seed,
    validation_split = 0.2,
    subset = "training",
    shuffle = True,
    class_names = class_names
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "training_data_final",
    labels = "inferred",
    label_mode = "categorical",
    color_mode = "rgb",
    batch_size = BATCH_SIZE,
    image_size = (96,96),
    seed = seed,
    validation_split = 0.2,
    subset = "validation",
    class_names = class_names
)

RESCALING AND DATA AUGMENTATION

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


In [ ]:
data_augmentation = keras.Sequential(
    [tfkl.RandomFlip("horizontal"), tfkl.RandomRotation(0.1), tfkl.RandomContrast(0.5)]
)

# Base Model


In [ ]:
input_shape = (96, 96, 3)
dropout_rate = 0.3
epochs = 1000

In [ ]:
def build_model(input_shape,dropout_rate):
    data_augmentation,

    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    x = tfkl.Rescaling(1./255)(input_layer)
    x = tfkl.Normalization()(x)

    x = tfkl.Conv2D(
        filters = 64,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'conv1')(x)

    x = tfkl.MaxPooling2D(name='mp1')(x)

    x = tfkl.Conv2D(
        filters = 128,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'conv2')(x)

    x = tfkl.MaxPooling2D(name='mp2')(x)

    x = tfkl.Conv2D(
        filters = 256,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'Conv3')(x)

    x = tfkl.GlobalAveragePooling2D(name='gap')(x)

    x = tfkl.Dropout(dropout_rate, seed=seed, name='gap_dropout')(x)
    x = tfkl.Dense(
        units = 256,
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'classifier')(x)

    x = tfkl.Dropout(dropout_rate, seed=seed, name='classifier_dropout')(x)
    x = tfkl.Dense(
        units = 256,
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'classifier1')(x)
    x = tfkl.Dropout(dropout_rate, seed=seed, name='classifier_dropout1')(x)
    output_layer = tfkl.Dense(
        units = 8,
        activation = 'softmax',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'output_layer')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs = input_layer, outputs = output_layer, name = 'model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
# Build model
normal_model = build_model(input_shape,dropout_rate,l1_lambda)
normal_model.summary()

In [ ]:
history = normal_model.fit(
    x = train_ds,
    epochs = epochs,
    validation_data = val_ds,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)],
).history

In [ ]:
normal_model.save("normal_model_080_normalization")

# XCEPTION


## TRANSFER LEARNING

In [ ]:
base_model = tfk.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3)
)
base_model.summary()

In [ ]:
# Use the supernet as feature extractor
base_model.trainable = False

inputs = keras.Input(shape=input_shape)
# Apply random data augmentation to the model
x = data_augmentation(inputs)
# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.)
scale_layer = keras.layers.Rescaling(scale = 1. /127.5, offset=-1)
x = scale_layer(x)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

x = tfkl.Dropout(dropout_rate, seed=seed, name='gap_dropout')(x)
x = tfkl.Dense(
        units = 256,
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'classifier')(x)

x = tfkl.Dropout(dropout_rate, seed=seed, name='classifier_dropout')(x)
x = tfkl.Dense(
        units = 256,
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'classifier1')(x)
x = tfkl.Dropout(dropout_rate, seed=seed, name='classifier_dropout1')(x)
output_layer = tfkl.Dense(
        units = 8,
        activation = 'softmax',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'output_layer')(x)

model_tl = tfk.Model(inputs = inputs, outputs = output_layer, name = 'tl_model')

model_tl.summary()

In [ ]:
model_tl.compile(
   loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy'
)

In [ ]:
# Train the model
tl_history = model_tl.fit(
    x = train_ds,
    batch_size = BATCH_SIZE,
    epochs = 500,
    validation_data = val_ds,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
model_tl.save('TransferLearningModelXception')

## FINE TUNE ALL THE MODEL

In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('TransferLearningModelXception')
ft_model.summary()

In [ ]:
# Set all layers to True
ft_model.trainable = True
for i, layer in enumerate(ft_model.layers):
   print(i, layer.name, layer.trainable)

In [ ]:
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

Starting from the weights from the transfer learning phase, retrain all the layers including the ones in the feature extraction part with a very low learning rate.

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = train_ds,
    batch_size = BATCH_SIZE,
    epochs = 500,
    validation_data = val_ds,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

In [ ]:
ft_model.save('FineTuningModelAllUnfreezed')

## FINE TUNING SOME UNFREEZED

Retrain the model freezing the first 95 layers and retraining only the dense part.

In [ ]:
# Re-load the model after transfer learning
ft_model_unfreezed = tfk.models.load_model('TransferLearningModelXception')
ft_model_unfreezed.summary()

In [ ]:
# Set all VGG layers to True
ft_model_unfreezed.get_layer('xception').trainable = True
for i, layer in enumerate(ft_model_unfreezed.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)

In [ ]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(ft_model_unfreezed.get_layer('xception').layers[:95]):
  layer.trainable=False
for i, layer in enumerate(ft_model_unfreezed.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)
ft_model_unfreezed.summary()

In [ ]:
ft_model_unfreezed.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [ ]:
# Fine-tune the model
ft_unfreezed_history = ft_model_unfreezed.fit(
    x = train_ds,
    batch_size = BATCH_SIZE,
    epochs = 500,
    validation_data = val_ds,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

In [ ]:
ft_model_unfreezed.save('FineTuningModelSomeUnfreezed')

# RESNET


## TRANSFER LEARNING

In [ ]:
#load ResNet50 model

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(180,180,3),
                   pooling='avg',classes=5,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

In [ ]:
def build_model():
  tf.keras.applications.resnet50.preprocess_input
  model = keras.Sequential()
  model.add(data_augmentation)
  model.add(tfkl.Resizing(180, 180, interpolation="bicubic"))
  model.add(pretrained_model)
  model.add(tfkl.Flatten())
  model.add(tfkl.Dense(
      units=256,
      activation='relu'))
  model.add(tfkl.BatchNormalization())
  model.add(tfkl.Dropout(rate=0.3, seed=seed))
  model.add(tfkl.Dense(8, activation='softmax'))
  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-2),
      loss="categorical_crossentropy",
      metrics=["accuracy"],
  )
  return model

In [ ]:
model = build_model()

In [ ]:
model.build(input_shape=(None, 96, 96, 3))
model.summary()

In [ ]:
#Retrain the model
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)
history = model.fit(
    x = train_ds,
    epochs = epochs,
    validation_data =val_ds,
    callbacks = [early_stopping],

).history

In [ ]:
model.save("TL_finale")

## FINE TUNING

In [ ]:
# Re-load the model after transfer learning
Resnet_ft_model = tfk.models.load_model('TL_finale')
Resnet_ft_model.summary()

In [ ]:
# Set all Resnet layers to True
Resnet_ft_model.get_layer('resnet50').trainable = True
for i, layer in enumerate(Resnet_ft_model.get_layer('resnet50').layers):
   print(i, layer.name, layer.trainable)

In [ ]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(Resnet_ft_model.get_layer('resnet50').layers[:142]):
  layer.trainable=False

for i, layer in enumerate(Resnet_ft_model.get_layer('resnet50').layers):
   print(i, layer.name, layer.trainable)
Resnet_ft_model.summary()

In [ ]:
# Compile the model
Resnet_ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [ ]:
# Fine-tune the model
Resnet_ft_history = Resnet_ft_model.fit(
    x = train_ds,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = val_ds,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
Resnet_ft_model.save("FT_finale")
del Resnet_ft_model

# KERAS TUNER

Using the tuner with different models, we have find the sub-optimal hyperparameters both for the straightforward model,for the xception model and for the resnet model.

In [ ]:
def build_model(hp):
  seed = 42
  random.seed(seed)
  model_kt = keras.Sequential()
  # data augmentation
  model_kt.add(tfkl.RandomFlip("horizontal") )
  model_kt.add(tfkl.RandomRotation(0.3))
  model_kt.add(tfkl.RandomContrast(0.5))

  model_kt.add(tfkl.Input(shape=(96,96,3), name='input_layer'))
  model_kt.add(tfkl.Rescaling(1./255))

  #convolutional part
  model_kt.add(tfkl.Conv2D(
        filters = 64,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'conv1'))
  model_kt.add(tfkl.MaxPooling2D(name='mp1'))

  model_kt.add(tfkl.Conv2D(
        filters = 128,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'conv2'))
  model_kt.add(tfkl.MaxPooling2D(name='mp2'))

  model_kt.add(tfkl.Conv2D(
        filters = 256,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'Conv3'))
  model_kt.add(tfkl.GlobalAveragePooling2D(name='gap'))
  #model_kt.add(tfkl.Flatten())

  # dense part
  for i in range(hp.Int("num_layers", 2, 4)):
      model_kt.add(
          tfkl.Dense(
              # Tune number of units separately.
              units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
              activation="relu",
          )
      )
      model_kt.add(tfkl.Dropout(rate=hp.Float("drop",min_value = 0.20,max_value = 0.50, step = 0.1)))

  model_kt.add(tfkl.Dense(
      units = 8,
      activation = 'softmax',
      kernel_initializer = tfk.initializers.GlorotUniform(seed),
      name = 'output_layer'))
  model_kt.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")), metrics='accuracy')

    # Return the model
  return model_kt

In [ ]:
build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = hp1.Hyperband(
   hypermodel = build_model,
   objective = "val_accuracy",
   max_epochs = 200,
   hyperband_iterations = 3,
   seed = seed
)

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(train_ds, epochs=10, validation_data=val_ds)